In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append('../../src')

In [4]:
import numpy as np
import pandas as pd
import data.database as db
import logging

In [15]:
DATASET_PATH = '../../datasets/amazon-sneakers'
DB_CONFIG = {
  'host'     : 'localhost',
  'user'     : 'root',
  'password' : '1234'
}

In [ ]:
database = db.Database(DATASET_PATH)

item_repo = db.ItemRepository(database)
interaction_repo = db.IteractionRepository(database)

In [12]:
items      = pd.read_json(f'{DATASET_PATH}/db_items.json')
interactions = pd.read_json(f'{DATASET_PATH}/db_interactions.json')

In [13]:
items.head(5)

,item_id,name,description,image
0,1,Superga Women's 2750 Cotu Classic Sneaker,None,https://m.media-amazon.com/images/I/612WcQ1+bI...
1,2,New Suede Shoe Cleaner Kit. Suede Brush & Sued...,None,https://m.media-amazon.com/images/I/71tbpuN6ZI...
2,3,New Balance Women's Dynasoft Nergize V3 Cross ...,None,https://m.media-amazon.com/images/I/71pR+J-gp4...
3,4,NCNDB Women's Camouflage Wedges Sneakers Inner...,welcome.our Sneaker shoes size tabs are chines...,https://m.media-amazon.com/images/I/61reE-na08...
4,5,Reebok Men's Classic Harman Run Sneaker,None,https://m.media-amazon.com/images/I/71FaYc6pSM...


In [14]:
interactions.head(5)

,user_id,item_id,rating
0,1,191,4.0
1,1,191,4.0
2,1,191,4.0
3,1,191,4.0
4,1,191,4.0


In [16]:
def execute(connection, query):
    try:
        cursor = connection.cursor()
        cursor.execute(query)
        connection.commit()
    except Exception as error:
        print(error)
        print(f'Query: {query}')


def insert_item(connection, row):
    query = """
        INSERT INTO 
            recsys.recsysweb_item(id, name, description, image)
        VALUES (
            :ID,
            ":NAME",
            ":DESC",
            ":IMAGE"
        );    
        """.replace(':ID', str(row['id'])) \
            .replace(':NAME', str(row['name'])) \
            .replace(':DESC', str(row['description']).replace('"', '')) \
            .replace(':IMAGE', str(row['image']))
    execute(connection, query)
        
def insert_interaction(connection, row):
    query = """
        INSERT INTO 
            recsys.recsysweb_interaction(item_id, user_id, rating)
        VALUES (
            ":ITEM_ID",
            ":USER_ID",
            ":RATING"
        );    
        """.replace(':ITEM_ID', str(int(row['item_id']))) \
            .replace(':USER_ID', str(int(row['user_id']))) \
            .replace(':RATING', str(row['rating']))
    execute(connection, query)

In [17]:
execute(connection, 'ALTER TABLE recsys.recsysweb_item MODIFY id int(11) NOT NULL')
execute(connection, 'DELETE FROM recsys.recsysweb_item')

1451 (23000): Cannot delete or update a parent row: a foreign key constraint fails 
(`recsys`.`recsysweb_interaction`, CONSTRAINT 
`recsysweb_interaction_item_id_818b6e1a_fk_recsysweb_item_id` FOREIGN KEY (`item_id`) 
REFERENCES `recsysweb_item` (`id`))

Query: DELETE FROM recsys.recsysweb_item

In [18]:
for _, row in movies.iterrows():
    insert_item(connection, row)    

NameError: name 'movies' is not defined

In [ ]:
execute(connection, 'DELETE FROM recsys.recsysweb_interaction')

In [ ]:
for _, row in interactions.iterrows():
    insert_interaction(connection, row)

In [ ]:
connection.close()

In [ ]:
interactions.user_id.max()